In [1]:
!pip install -q google-cloud-aiplatform pytest


Import the required packages

In [2]:
from vertexai.language_models import ChatModel, InputOutputTextPair
from vertexai.preview.language_models import TextGenerationModel
import vertexai

Configure the project

In [3]:
PROJECT_ID = "qwiklabs-gcp-02-4c9c7fb5e8ec"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)


Create the LLM model

In [4]:
from vertexai.preview.generative_models import GenerativeModel
chat_model = GenerativeModel("gemini-2.0-flash-001")

Python function that uses Gemini to classify the question to specific categories

In [5]:
def classify_question_category(question: str) -> str:
    prompt = f"""You are a bot trained to categorize the any type of questions to either Employment,General Information, Emergency Services or Tax Related.
    if the questions can't be categorized into any of this, Respond the user with "Sorry. Your question is not be classified under Employment, General Information, Emergency Services, or Tax Related."
    The classification should be only of : "Employment", "General Information", "Emergency Services", "Tax Related". or with the error prompt.

    Question: {question}"""

    response = chat_model.generate_content(prompt)
    return response.text.strip()

classify_question_category("What is the tax on roads?")

'Tax Related'

Testcase for the question classification

In [6]:
import unittest
class test_classification(unittest.TestCase):
    def test_Employment(self):
      category = classify_question_category("What is the current employment utilzation percentil?")
      self.assertEqual(category, "Employment")

    def test_General_Information(self):
      category = classify_question_category("Is the government work is more related to public?")
      self.assertEqual(category, "General Information")

    def test_Emergency_Services(self):
      category = classify_question_category("what is the emergency number for hospitals?")
      self.assertEqual(category, "Emergency Services")

    def test_Tax_Related(self):
      category = classify_question_category("what is the tax on the gold?")
      self.assertEqual(category, "Tax Related")


unittest.main(argv=[''], verbosity=1, exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 1.141s

OK


Generate the social post using Python function

In [7]:


def generate_social_post(event: str) -> str:
    """
    Generates a social media post about a government event, ensuring it's under 280 characters.
    """
    prompt = f"""
    You are a helpful government assistant. Write a short, friendly, informative social media post (under 280 characters) about this announcement:
    Event: {event}
    """

    response = chat_model.generate_content(prompt)
    return response.text.strip()



 Evaluate the Post generated by Python

In [8]:


def eval_generate_post(event: str) -> str:
    """
    Evaluates a generated post based on criteria. Returns rating as a number (string).
    """
    system_prompt = (
        f"""
        You are announcement design reviewer for a government agency.
        You are asked to evaluate the post for public announcements based on the criteria of Clarity, Tone, Accuracy, Urgency, and Format.
        Rate the generated post on a scale of 1 to 5 for clarity, relevance, tone, and informativeness. give the 2 hastags for the post that is generated
        Just return the number only.
        """
    )

    generated_post = generate_social_post(event)

    # Feed generated_post to reviewer (LLM)
    eval_prompt = f"{system_prompt}\nPost: {generated_post}"

    response = chat_model.generate_content(eval_prompt)
    return response.text.strip()

Unit testcase to evaluate the post generated by the LLM

In [9]:
import unittest

class TestEvalGeneratePost(unittest.TestCase):

    def test_eval_generate_post(self):
        result = eval_generate_post("Say No to Single-Use Plastic!")
        print(result)

unittest.main(argv=[''], exit=False)


.

Clarity: 5
Tone: 5
Accuracy: 5
Urgency: 3

#SayNoToPlastic #Sustainability


....
----------------------------------------------------------------------
Ran 5 tests in 1.967s

OK


Evaluation API

Create an example dataset for the evaluation API

In [10]:
example_dataset = [
    {
        "prompt": "A new metro line will be launched in Mumbai on August 20, connecting Versova to Andheri West.",
        "post": "Exciting news, Mumbai! 🚇 A new metro line opens on August 20, linking Versova to Andheri West. Smooth rides ahead! #MumbaiMetro #CityTransportation"
    },
    {
        "prompt": "The local zoo in Jaipur is hosting a special wildlife conservation event from July 5-7, featuring workshops and animal shows.",
        "post": "Join us at the Jaipur Zoo from July 5-7 for the Wildlife Conservation event! 🦁🐒 Workshops, animal shows, and more. A great day out! #JaipurZoo #WildlifeConservation"
    },
    {
        "prompt": "The district court in Pune will remain closed on September 30 for maintenance work.",
        "post": "Heads up, Pune! The district court will be closed on September 30 for maintenance. Plan your visit accordingly. #PuneCourt #CourtClosure"
    },
    {
        "prompt": "New traffic rules will be enforced in Chandigarh starting July 1, with increased fines for violations.",
        "post": "🚗🚨 New traffic rules in Chandigarh from July 1! Be cautious as fines for violations are set to increase. Drive safely! #ChandigarhTraffic #NewRules"
    },
    {
        "prompt": "Due to a public event, the area around Connaught Place in New Delhi will be closed to traffic on June 25 from 10 AM to 4 PM.",
        "post": "New Delhi, Connaught Place is closed to traffic on June 25 from 10 AM to 4 PM for a public event. Plan your routes in advance! 🚧 #DelhiTraffic #RoadClosure"
    },
    {
        "prompt": "A special vaccination camp for children will be organized in Pune on July 12 at the community center.",
        "post": "Pune parents, mark your calendars! 🗓️ A vaccination camp for kids will be held on July 12 at the community center. Keep your children safe! #PuneVaccination #ChildHealth"
    },
    {
        "prompt": "The airport in Bengaluru will be conducting a security drill on August 1. Expect delays in check-in and security checks.",
        "post": "Heads up, Bengaluru travelers! 🛫 A security drill at the airport on August 1 means possible delays. Arrive early! #BengaluruAirport #TravelTips"
    },
    {
        "prompt": "The famous food festival in Lucknow will be held from September 10-12, showcasing local delicacies and street food.",
        "post": "Get your taste buds ready! 🍴 The Lucknow Food Festival is happening from September 10-12. Enjoy local flavors and street food! #LucknowFoodFestival #FoodLovers"
    },
    {
        "prompt": "The local library in Surat will be offering free reading sessions for children every Saturday throughout the summer.",
        "post": "Surat parents, get your kids into reading! 📚 Free reading sessions for children every Saturday this summer at the local library. #SuratLibrary #SummerFun"
    },
    {
        "prompt": "The state government has announced a ban on the sale of fireworks in all major cities from October 15 to November 15.",
        "post": "Attention, citizens! 🚫 Fireworks sale will be banned in all major cities from October 15 to November 15. Stay safe this festive season! #FireworksBan #DiwaliSafety"
    },
    {
        "prompt": "The famous Kumbh Mela in Haridwar will take place from January 14-28, attracting millions of visitors from across India.",
        "post": "Kumbh Mela 2024 in Haridwar is around the corner! 🕉️ From January 14-28, experience one of the world's largest religious gatherings. #KumbhMela #Haridwar"
    },
    {
        "prompt": "A local marathon will be held in Chennai on August 25. Roads in the city center will be closed from 5 AM to 11 AM.",
        "post": "Chennai runners, get ready for the marathon on August 25! 🏃‍♀️ Roads in the city center will be closed from 5 AM to 11 AM. #ChennaiMarathon #RoadClosure"
    },
    {
        "prompt": "The railway station in Varanasi will be undergoing major renovations starting August 1. Expect crowd management and limited facilities.",
        "post": "Travelers, please note: Varanasi Railway Station will be under renovation from August 1. Be prepared for crowd management and limited services. #VaranasiStation #RailwayRenovation"
    },
    {
        "prompt": "There will be a temporary shutdown of the water supply in Sector 15, Noida on July 8 from 9 AM to 5 PM.",
        "post": "No water in Sector 15, Noida on July 8 from 9 AM to 5 PM! 💧 Plan ahead and stock up. #NoidaWaterShutdown #Sector15"
    },
    {
        "prompt": "The local museum in Bhopal will be closed for maintenance work from July 10-15. No entry during these dates.",
        "post": "Bhopal Museum will be closed for maintenance from July 10-15. Plan your visit accordingly! 🖼️ #BhopalMuseum #Maintenance"
    }
]


Creating an evaluation dataset

In [11]:

import pandas as pd

eval_dataset = pd.DataFrame([
    {
        "instruction": f"Generates a social media post about a government event, ensuring it's under 280 characters.",
        "prompt": f"You are a helpful government assistant. Write a short, friendly, informative social media post (under 280 characters) about this announcement:\nEvent: {item['prompt']}",
        "context": f"announcement: {item['prompt']}",
        "response": item["post"],
    }
    for item in example_dataset
])

Creating an evaluation task

In [12]:
import datetime
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"social-media-post-{run_ts}"
)

Run the evaluation

In [13]:
prompt_template = (
    "Instruction: {instruction}. Prompt: {context}. Post: {response}"
)
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"social-media-post-{run_ts}"
)
evaluation_results = []
evaluation_results.append(result)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Prompt: {context}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 60 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 60/60 [00:06<00:00,  9.29it/s]
INFO:vertexai.evaluation._evaluation:All 60 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:6.4842598379998435 seconds


Examine the results

In [14]:
from vertexai.preview.evaluation import notebook_utils
notebook_utils.display_eval_result(eval_result=result)

### Summary Metrics

,row_count,groundedness/mean,groundedness/std,verbosity/mean,verbosity/std,instruction_following/mean,instruction_following/std,safety/mean,safety/std
0,15.0,1.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0


### Row-based Metrics

,instruction,prompt,context,response,groundedness/explanation,groundedness/score,verbosity/explanation,verbosity/score,instruction_following/explanation,instruction_following/score,safety/explanation,safety/score
0,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: A new metro line will be launche...,"Exciting news, Mumbai! 🚇 A new metro line open...",The response is entirely grounded in the provi...,1.0,"The response is appropriately concise, providi...",0.0,The response perfectly follows the instruction...,5.0,The response is safe as it does not contain an...,1.0
1,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: The local zoo in Jaipur is hosti...,Join us at the Jaipur Zoo from July 5-7 for th...,The generated social media post is entirely gr...,1.0,"The response is perfectly concise, providing a...",0.0,The AI's response precisely followed the user'...,5.0,The response is safe as it does not contain an...,1.0
2,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: The district court in Pune will ...,"Heads up, Pune! The district court will be clo...",The generated social media post is entirely ba...,1.0,"The response is perfectly concise, providing a...",0.0,The response perfectly followed the instructio...,5.0,The response is safe as it does not contain an...,1.0
3,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: New traffic rules will be enforc...,🚗🚨 New traffic rules in Chandigarh from July 1...,The AI model successfully generated a response...,1.0,"The response is perfectly concise, providing a...",0.0,The AI model accurately generated a social med...,5.0,The response is safe as it does not contain an...,1.0
4,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,"announcement: Due to a public event, the area ...","New Delhi, Connaught Place is closed to traffi...",The response accurately uses information provi...,1.0,"The response is perfectly concise, providing a...",0.0,The response completely adheres to the prompt'...,5.0,The response is safe as it does not contain an...,1.0
5,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: A special vaccination camp for c...,"Pune parents, mark your calendars! 🗓️ A vaccin...","The response is fully grounded, as all aspects...",1.0,The response is perfectly concise and provides...,0.0,The AI model completely followed the instructi...,5.0,The response is safe because it does not conta...,1.0
6,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: The airport in Bengaluru will be...,"Heads up, Bengaluru travelers! 🛫 A security dr...",The AI-generated response is fully grounded in...,1.0,The response is perfectly concise and provides...,0.0,The AI model has completely fulfilled the inst...,5.0,The response is safe as it does not contain an...,1.0
7,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: The famous food festival in Luck...,Get your taste buds ready! 🍴 The Lucknow Food ...,The response is completely grounded in the pro...,1.0,"The response is perfectly concise, providing a...",0.0,The response perfectly followed the instructio...,5.0,The response is safe because it does not conta...,1.0
8,Generates a social media post about a governme...,Instruction: Generates a social media post abo...,announcement: The local library in Surat will ...,"Surat parents, get your kids into reading! 📚 F...",The response is fully grounded in the user pro...,1.0,"The response is perfectly concise, providing a...",0.0,"The response perfectly matches the prompt, ful...",5.0,The response is safe as it does not contain an.